In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts
import warnings
import datetime
import time
#import threading
import concurrent.futures
#import multiprocessing
import requests

In [234]:
from datetime import timedelta
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import coint
#from sklearn.cluster import DBSCAN
#from sklearn.cluster import KMeans
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from itertools import combinations
from scipy.stats import spearmanr
from sklearn.linear_model import LinearRegression
from multiprocessing import Pool, Process, Queue
from pyswarm import pso
from dask.distributed import Client, as_completed
import dask

In [806]:
def get_sp500_tickers():
    # Ссылка на страницу Wikipedia с перечнем компаний S&P 500
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

    # Скачивание таблицы с помощью pandas
    sp500_table = pd.read_html(url, header=0)[0]
    tickers = sp500_table['Symbol'].tolist()

    # Обработка символов, которые могут вызвать проблемы (например, замена '.' на '-')
    tickers = [ticker.replace('.', '-') for ticker in tickers]

    return tickers

# Получение списка тикеров
tickers = get_sp500_tickers()

start_date = "2020-01-02"
end_date = "2023-02-02"
data = yf.download(tickers, start=start_date, end=end_date)['Close']
data_open = yf.download(tickers, start=start_date, end=end_date)['Open']

[*********************100%%**********************]  503 of 503 completed

2 Failed downloads:
['VLTO', 'KVUE']: Exception("%ticker%: Data doesn't exist for startDate = 1577941200, endDate = 1675314000")
[*********************100%%**********************]  503 of 503 completed

2 Failed downloads:
['VLTO', 'KVUE']: Exception("%ticker%: Data doesn't exist for startDate = 1577941200, endDate = 1675314000")


In [ ]:
# Загрузка котировок с Yahoo
data_2022 = yf.download(unique_tickers, start="2022-01-02", end="2023-02-02")['Close']
data_2022_open = yf.download(unique_tickers, start="2022-01-02", end="2023-02-02")['Open']
data_2m = yf.download(unique_tickers, start="2022-11-02", end="2023-02-02")['Close']
data_2m_open = yf.download(unique_tickers, start="2022-11-02", end="2023-02-02")['Open']
data_6m = yf.download(unique_tickers, start="2022-07-02", end="2023-02-02")['Close']
data_6m_open = yf.download(unique_tickers, start="2022-07-02", end="2023-02-02")['Open']
data_2023 = yf.download(unique_tickers, start="2023-01-02", end="2024-02-02")['Close']
data_2023_open = yf.download(unique_tickers, start="2023-01-02", end="2024-02-02")['Open']

In [893]:
data_2023_2 = yf.download(unique_tickers, start="2023-02-02", end="2023-06-02")['Close']
data_2023_2_open = yf.download(unique_tickers, start="2023-02-02", end="2023-06-02")['Open']

[*********************100%%**********************]  286 of 286 completed
[*********************100%%**********************]  286 of 286 completed


In [808]:
# Удаляем столбцы, где более 10% данных отсутствуют
threshold = 0.1  # 10% порог
data = data.dropna(thresh=int((1 - threshold) * len(data)), axis=1)

# Заполнение оставшихся пропусков, например, методом ffill (forward fill)
data = data.ffill()

# Удаляем строки, которые все еще содержат NaN (если они есть)
data = data.dropna()

# Удаляем тикеры с низкой волатильностью
for tiker in tickers:
    if tiker in data:
        std_tiker = data_6m[tiker].std()
        if std_tiker < 7:
            data = data.drop(tiker, axis=1)
len(data.columns)

286

In [690]:
# Удаляем столбцы, где более 10% данных отсутствуют
threshold = 0.1  # 10% порог
data_2022 = data_2022.dropna(thresh=int((1 - threshold) * len(data_2022)), axis=1)
# Заполнение оставшихся пропусков, например, методом ffill (forward fill)
data_2022 = data_2022.ffill()
# Удаляем строки, которые все еще содержат NaN (если они есть)
data_2022 = data_2022.dropna()

In [809]:
unique_tickers = list(data.columns)
num_tickers = len(unique_tickers)

In [812]:
pairs = list(combinations(data.columns, 2))
pairs_filter = []

def tests(pair):
    # Характеристики пар
    # Фильтр пар
    tik1, tik2 = pair
    adf = 0
    if tik1 in data.columns and tik2 in data.columns:
        stock1 = data[tik1]
        stock2 = data[tik2]
        # Рассчитываем коинтеграцию с помощью CADF теста
        score, p_value, _ = ts.coint(stock1, stock2)
        # ADF тест
        spread = np.log(stock1) - np.log(stock2)
        adf_result = ts.adfuller(spread)
        # Корреляция Спирмена
        corr, spear_p = spearmanr(stock1, stock2)
        if adf_result[0] < adf_result[4]['1%']:
            adf = 1
    return {'pair' : pair, 'ADF' : adf, 'ADF-stat' : adf_result[0], 'CADF-score' : score, 'Spearman' : corr}

pairs_filter = Parallel(n_jobs=12)(delayed(tests)(pair) for pair in pairs)
pairs_filter = pd.DataFrame(pairs_filter)
pairs_filter.head()

,pair,ADF,ADF-stat,CADF-score,Spearman
0,"(A, AAPL)",0,-2.234990,-1.669445,0.703878
1,"(A, ABBV)",0,-1.525241,-1.830188,0.575407
2,"(A, ACGL)",0,-1.590776,-1.776220,0.645376
3,"(A, ACN)",0,-1.507011,-1.233028,0.852876
4,"(A, ADBE)",0,-0.637042,-1.106030,0.447988


In [839]:
# Фильтр пар
filtred_pairs = []
for i in range(len(pairs_filter)):
    if pairs_filter.at[i, 'ADF'] == 1:
       filtred_pairs.append(pairs_filter.at[i, 'pair'])
        # if pairs_filter.at[i, 'Spearman'] >= 0.5 and pairs_filter.at[i, 'CADF-score'] < -3:
        #     filtred_pairs.append(pairs_filter.at[i, 'pair'])
len(filtred_pairs)

1592

In [706]:
# Вспомогательные датафреймы
line_reg = data.copy()
deviation = data.copy()
relative_deviation = data.copy()
spread_pairs = data.copy()
mean_price = data.copy()
slope = data.copy()
line_reg_long = data.copy()
slope_long = data.copy()
line_reg_long.loc[:, :] = np.nan
slope_long.loc[:, :] = np.nan
slope.loc[:, :] = np.nan
mean_price.loc[:, :] = np.nan
spread_pairs.loc[:, :] = np.nan
line_reg.loc[:, :] = np.nan
deviation.loc[:, :] = np.nan
relative_deviation.loc[:, :] = np.nan


In [61]:
def send_telegram_message(message):
    # Параметры для отправки сообщения
    telegram_token = '6737079836:AAHQT4Bgz3Kh8LRA_kaQDJ76HJBzh9S_X8A'
    chat_id = '264028245'
    url = f'https://api.telegram.org/bot{token}/sendMessage'
    payload = {
        'chat_id': chat_id,
        'text': message,
        'parse_mode': 'Markdown'
    }
    response = requests.post(url, json=payload)
    return response.json()

In [729]:
def regression(tiker):
    # Подготовка данных
    data['DateNumeric'] = data.index.map(pd.Timestamp.toordinal)
    roll_window = 12
    window_regression = 6

    # Локальные структуры данных для хранения результатов
    line_reg_loc = pd.Series(index=data.index, dtype='float64')
    relative_deviation_loc = pd.Series(index=data.index, dtype='float64')
    line_reg_long_loc = pd.Series(index=data.index, dtype='float64')
    slope_long_loc = pd.Series(index=data.index, dtype='float64')

    # Создание признаков для модели
    X = data['DateNumeric'].values.reshape(-1, 1)

    for index in range(roll_window, len(data)):
        current_date = data.index[index]
        tiker_price = data.at[current_date, tiker]

        # Расчет отклонения от линии регрессии котировок
        numeric_date = current_date.toordinal()

        # Рассчет для коротких позиций
        if index >= roll_window:
            model = LinearRegression()
            model.fit(X[index-roll_window:index], data[tiker][index-roll_window:index])
            predicted_price = model.predict([[numeric_date]])
            line_reg_loc[current_date] = predicted_price[0]
            relative_deviation_loc[current_date] = (tiker_price - predicted_price) / predicted_price

        # Расчет для длинных позиций
        if index >= window_regression:
            model_long = LinearRegression()
            model_long.fit(X[index-window_regression:index], data[tiker][index-window_regression:index])
            predicted_price_long = model_long.predict([[numeric_date]])
            line_reg_long_loc[current_date] = predicted_price_long[0]
            slope_long_loc[current_date] = model_long.coef_[0]

    # Формирование результатов для возврата
    results = {
        'line_reg': line_reg_loc,
        'relative_deviation': relative_deviation_loc,
        'line_reg_long': line_reg_long_loc,
        'slope_long': slope_long_loc,
    }

    return results

In [730]:
regres_results = Parallel(n_jobs=-1)(delayed(regression)(tiker) for tiker in unique_tickers)
for tiker, results in zip(unique_tickers, regres_results):
    # Результаты для каждого тикера
    line_reg_results = results['line_reg']
    relative_deviation_results = results['relative_deviation']
    line_reg_long_results = results['line_reg_long']
    slope_long_results = results['slope_long']
    
    # Обновление соответствующих датафреймов
    line_reg[tiker] = line_reg_results
    relative_deviation[tiker] = relative_deviation_results
    line_reg_long[tiker] = line_reg_long_results
    slope_long[tiker] = slope_long_results

In [740]:
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import pandas as pd

# Функция для расчета линейной регрессии с возможностью настройки параметров
def regression_with_params(data, tiker, roll_window):
    data['DateNumeric'] = data.index.map(pd.Timestamp.toordinal)
    
    line_reg_loc = pd.Series(index=data.index, dtype='float64')
    relative_deviation_loc = pd.Series(index=data.index, dtype='float64')
    
    X = data['DateNumeric'].values.reshape(-1, 1)
    y_true = []
    y_pred = []

    for index in range(roll_window, len(data)-2):
        current_date = data.index[index+1]
        tiker_price = data.at[current_date, tiker]
        numeric_date = current_date.toordinal()

        if index >= roll_window:
            model = LinearRegression()
            model.fit(X[index-roll_window:index], data[tiker][index-roll_window:index])
            predicted_price = model.predict([[numeric_date]])
            line_reg_loc[current_date] = predicted_price[0]
            relative_deviation_loc[current_date] = (tiker_price - predicted_price) / predicted_price
            y_true.append(tiker_price)
            y_pred.append(predicted_price[0])

    mse = mean_squared_error(y_true, y_pred)
    return mse

def optimize_roll_window(data, tiker):
    best_mse = np.inf
    best_roll_window = None

    for roll_window in range(2, 20):  # Пример диапазона для roll_window
        mse = regression_with_params(data, tiker, roll_window)
        if mse < best_mse:
            best_mse = mse
            best_roll_window = roll_window
    
    return best_roll_window, best_mse#ANET, ROK

In [743]:

# Пример использования
tiker = ('PSX')
fixed_window_regression = 8
# best_params, best_mse = optimize_roll_window(data, tiker, fixed_window_regression)
# Задайте начальное предположение для window_regression

# Оптимизируйте roll_window
best_roll_window, mse_for_best_roll_window = optimize_roll_window(data, tiker)
print(f"Лучший roll_window: {best_roll_window}, MSE: {mse_for_best_roll_window}")

Лучший roll_window: 6, MSE: 15.697276446115254


In [787]:
def pairs_trading(pair, close_data, open_data, options):
    # Тестируемые тикеры акций
    name_1, name_2 = pair

    ticker1 = close_data[name_1]
    ticker2 = close_data[name_2]

    # Начальные параметры
    initial_capital = 100000    # Стартовый капитал
    max_cash = initial_capital    # Максимально достигнутая сумма
    cash = initial_capital    # Баланс кошелька
    low_shares = 0    # Акции low
    jbht_shares = 0    # Акции jbht
    trade_count = 0   # Счетчик сделок
    line_count = 0  # Счетчик сделок по линейной регрессии
    window = options[0]  # Окно для скользящего среднего и стандартного отклонения
    window_regression = options[1]
    coef_long = options[2]
    roll_window = options[3]  # Окно для линейной регрессии
    coef_entry = options[4]
    coef_sell = options[5]
    coef_loss = options[6]
    coef_profit = options[7]
    #coef_short = options[8]

    flag = True # Индикатор стоп-лосса
    long_position = False # Идикатор длинной позиции
    #short_position = False # Идикатор короткой позиции

    # Объединение данных по дате
    z_test = pd.merge(ticker1, ticker2, left_index=True, right_index=True, how='inner')
    z_test.columns = ['JBHT_Close', 'LOW_Close']

    # Расчет логарифмической разности
    z_test['Spread'] = np.log(z_test['LOW_Close']) - np.log(z_test['JBHT_Close'])
    # Расчет z-score
    z_test['Mean_spread'] = z_test['Spread'].rolling(window=window).mean()
    z_test['Std_spread'] = z_test['Spread'].rolling(window=window).std()
    z_test['Z-score'] = (z_test['Spread'] - z_test['Mean_spread']) / z_test['Std_spread']
    #z_test['JBHT_mean'] = z_test['JBHT_Close'].rolling(window=10).mean()
    #z_test['LOW_mean'] = z_test['LOW_Close'].rolling(window=10).mean()
    #z_test['JBHT_std'] = z_test['JBHT_Close'].rolling(window=10).std()
    #z_test['LOW_std'] = z_test['LOW_Close'].rolling(window=10).std()

    for index in range(window, len(z_test)-1):

        current_date = z_test.index[index]
        numeric_date = current_date.toordinal()
        open_date = z_test.index[index+1]
        jbht_close = z_test.at[current_date, 'JBHT_Close']
        low_close = z_test.at[current_date, 'LOW_Close']
        jbht_price = open_data.at[open_date, name_1]
        low_price = open_data.at[open_date, name_2]

        # Расчет скользящего среднего и стандартного отклонения для спреда
        std_spread = z_test.at[current_date, 'Std_spread']
        #mean_spread = z_test.at[current_date, 'Mean_spread']
        z_score = z_test.at[current_date, 'Z-score']

        # predicted_jbht = line_reg.at[current_date, name_1]
        # predicted_low = line_reg.at[current_date, name_2]

        predicted_jbht_long = line_reg_long.at[current_date, name_1]
        predicted_low_long = line_reg_long.at[current_date, name_2]
        jbht_slope = slope_long.at[current_date, name_1]  # Коэффициент наклона
        low_slope = slope_long.at[current_date, name_2]  # Коэффициент наклона

        # Отклонение цены акции от линии регрессии для Z-score
        deviation_jbht = relative_deviation.at[current_date, name_1]
        deviation_low = relative_deviation.at[current_date, name_2]

        # Устанавливаем пороги на основе волатильности
        entry = coef_entry * std_spread
        sell = coef_sell * std_spread

        # if flag == False:
        #     entry = 3 * coef_entry * std_spread
        #     sell = 3 * coef_sell * std_spread            
        #     if index > loss_date + 2:
        #         flag = True
        #         entry = coef_entry * std_spread
        #         sell = coef_sell * std_spread

        # Торговые сигналы и сделки
        # Покупка спреда
        # if flag:
        # Покупка спреда при восходящей регрессии двух тикеров
        # if jbht_close > predicted_jbht_long and low_close > predicted_low_long and low_slope > coef_long and jbht_slope > coef_long:
        #     if low_shares == 0 and jbht_shares == 0:
        #         low_shares = 0.5 * cash / low_price
        #         jbht_shares = 0.5 * cash / jbht_price
        #         cash -= low_shares * low_price
        #         cash -= jbht_shares * jbht_price
        #         line_count += 1
        #     elif low_shares < 0 and jbht_shares > 0:
        #         cash += low_shares * low_price
        #         low_shares = 0
        #         line_count += 1
        #         jbht_shares += cash / jbht_price
        #         cash = 0
        #     elif low_shares > 0 and jbht_shares < 0:
        #         cash += jbht_shares * jbht_price
        #         jbht_shares = 0
        #         line_count += 1
        #         low_shares += cash / low_price
        #         cash = 0
        #     long_position = True

        # elif (jbht_close < predicted_jbht_long or low_close < predicted_low_long) and long_position == True:
        #     cash += jbht_shares * jbht_price
        #     cash += low_shares * low_price
        #     jbht_shares = 0
        #     low_shares = 0
        #     long_position = False
        #     line_count += 1

        # Стратегия z-счет
        if z_score < -entry and long_position == False:
            # Длинная позиция по дорогой, короткая по дешевой
            if deviation_low < deviation_jbht:
                if low_shares == 0 and jbht_shares == 0:
                    low_shares = cash / low_price
                    jbht_shares = -(cash / jbht_price)
                    cash -= low_shares * low_price
                    cash += -jbht_shares * jbht_price
                    trade_count += 2
                elif low_shares > 0 and jbht_shares < 0:
                    delta = (-cash / jbht_price - jbht_shares) * jbht_price
                    low_shares -= delta / low_price
                    jbht_shares = -(cash / jbht_price)
                    trade_count += 1
                elif low_shares < 0 and jbht_shares > 0:
                    cash += jbht_shares * jbht_price
                    cash += low_shares * low_price
                    low_shares = 0
                    jbht_shares = 0
                    low_shares = cash / low_price
                    jbht_shares = -(cash / jbht_price)
                    trade_count += 2
            # Короткая позиция по LOW, длинная по JBHT
            else: 
                if low_shares == 0 and jbht_shares == 0:
                    jbht_shares = cash / jbht_price
                    low_shares = -(cash / low_price)
                    cash -= jbht_shares * jbht_price
                    cash += -low_shares * low_price
                    trade_count += 2
                elif low_shares < 0 and jbht_shares > 0:
                    delta = (-cash / low_price - low_shares) * low_price
                    jbht_shares -= delta / jbht_price
                    low_shares = -(cash / low_price)
                    trade_count += 1
                elif low_shares > 0 and jbht_shares < 0:
                    cash += low_shares * low_price
                    cash += jbht_shares * jbht_price
                    low_shares = 0
                    jbht_shares = 0
                    jbht_shares = cash / jbht_price
                    low_shares = -(cash / low_price)
                    trade_count += 2

        # Покупка спреда
        elif z_score > entry and long_position == False:
            # Короткая позиция по LOW, длинная по JBHT
            if deviation_jbht < deviation_low:
                if low_shares == 0 and jbht_shares == 0:
                    jbht_shares = cash / jbht_price
                    low_shares = -(cash / low_price)
                    cash -= jbht_shares * jbht_price
                    cash -= low_shares * low_price
                    trade_count += 2
                elif low_shares < 0 and jbht_shares > 0:
                    delta = (-cash / low_price - low_shares) * low_price
                    jbht_shares -= delta / jbht_price
                    low_shares = -(cash / low_price)
                    trade_count += 1
                elif low_shares > 0 and jbht_shares < 0:
                    cash += low_shares * low_price
                    cash += jbht_shares * jbht_price
                    low_shares = 0
                    jbht_shares = 0
                    jbht_shares = cash / jbht_price
                    low_shares = -(cash / low_price)
                    trade_count += 2
            # Длинная позиция по LOW, короткая по JBHT
            else:
                if low_shares == 0 and jbht_shares == 0:
                    low_shares = cash / low_price
                    jbht_shares = -(cash / jbht_price)
                    cash -= low_shares * low_price
                    cash += -jbht_shares * jbht_price
                    trade_count += 2
                elif low_shares > 0 and jbht_shares < 0:
                    delta = (-cash / jbht_price - jbht_shares) * jbht_price
                    low_shares -= delta / low_price
                    jbht_shares = -(cash / jbht_price)
                    trade_count += 1
                elif low_shares < 0 and jbht_shares > 0:
                    cash += jbht_shares * jbht_price
                    cash += low_shares * low_price
                    low_shares = 0
                    jbht_shares = 0
                    low_shares = cash / low_price
                    jbht_shares = -(cash / jbht_price)
                    trade_count += 2

        # Выход из позиции по спреду
        elif -sell < z_score < sell and low_shares != 0 and long_position == False:
            cash += low_shares * low_price
            cash += jbht_shares * jbht_price
            low_shares = 0
            jbht_shares = 0
            trade_count += 2

        # Установка стоп-лосса
        # Расчет текущей стоимости позиций         
        value_low = low_shares * low_close
        value_jbht = jbht_shares * jbht_close
        total_value = cash + value_low + value_jbht

        if max_cash < cash:
            max_cash = cash
        stop_loss = coef_loss * max_cash

        # Расчет текущей позиции
        current_loss = max_cash - total_value

        # Проверка условий стоп-лосса
        if current_loss > stop_loss:
            # Закрытие всех позиций
            cash += low_shares * low_price
            cash += jbht_shares * jbht_price
            low_shares = 0
            jbht_shares = 0
            trade_count += 1
            max_cash = cash
            # flag = False
            # loss_date = index
                
        # Установка take-profit
        stop_profit = coef_profit * max_cash

        # Расчет текущего профита
        current_profit = total_value - max_cash

        if current_profit > stop_profit:
            # Закрытие всех позиций
            cash += low_shares * low_price
            cash += jbht_shares * jbht_price
            low_shares = 0
            jbht_shares = 0
            trade_count += 1

        # Если есть шорт позиции, списываем комиссию за день
        if jbht_shares < 0:
            daily_fee = -jbht_shares * jbht_price * 0.13 / 365
            cash -= daily_fee
        elif low_shares < 0:
            daily_fee = -low_shares * low_price * 0.13 / 365
            cash -= daily_fee
            
        #cash[pair] = total_value
                   
    return {'pair': pair, 'final_result': cash, 'Position' : total_value, 'trade_count': trade_count, 'line_count' : line_count}

In [836]:
def pairs_trading_heavy(pair, close_data, open_data, options):
    # Тестируемые тикеры акций
    name_1, name_2 = pair

    ticker1 = close_data[name_1]
    ticker2 = close_data[name_2]

    # Начальные параметры
    initial_capital = 100000    # Стартовый капитал
    max_cash = initial_capital    # Максимально достигнутая сумма
    cash = initial_capital    # Баланс кошелька
    low_shares = 0    # Акции low
    jbht_shares = 0    # Акции jbht
    trade_count = 0   # Счетчик сделок
    line_count = 0  # Счетчик сделок по линейной регрессии
    window = options[0]  # Окно для скользящего среднего и стандартного отклонения
    window_regression = options[1]
    coef_long = options[2]
    roll_window = options[3]  # Окно для линейной регрессии
    coef_entry = options[4]
    coef_sell = options[5]
    coef_loss = options[6]
    coef_profit = options[7]
    #coef_short = options[8]

    flag = True # Индикатор стоп-лосса
    long_position = False # Идикатор длинной позиции
    #short_position = False # Идикатор короткой позиции

    # Объединение данных по дате
    z_test = pd.merge(ticker1, ticker2, left_index=True, right_index=True, how='inner')
    z_test.columns = ['JBHT_Close', 'LOW_Close']

    # Расчет логарифмической разности
    z_test['Spread'] = np.log(z_test['LOW_Close']) - np.log(z_test['JBHT_Close'])
    # Расчет z-score
    z_test['Mean_spread'] = z_test['Spread'].rolling(window=window).mean()
    z_test['Std_spread'] = z_test['Spread'].rolling(window=window).std()
    z_test['Z-score'] = (z_test['Spread'] - z_test['Mean_spread']) / z_test['Std_spread']
    z_test['DateNumeric'] = z_test.index.map(pd.Timestamp.toordinal)        # Перевод дат в цифровой формат
    X = z_test['DateNumeric'].values.reshape(-1, 1)     # Создание индексов в качестве признаков
    #z_test['JBHT_mean'] = z_test['JBHT_Close'].rolling(window=10).mean()
    #z_test['LOW_mean'] = z_test['LOW_Close'].rolling(window=10).mean()
    #z_test['JBHT_std'] = z_test['JBHT_Close'].rolling(window=10).std()
    #z_test['LOW_std'] = z_test['LOW_Close'].rolling(window=10).std()

    for index in range(window, len(z_test)-1):

        current_date = z_test.index[index]
        numeric_date = current_date.toordinal()
        open_date = z_test.index[index+1]
        jbht_close = z_test.at[current_date, 'JBHT_Close']
        low_close = z_test.at[current_date, 'LOW_Close']
        jbht_price = open_data.at[open_date, name_1]
        low_price = open_data.at[open_date, name_2]
        #jbht_predict = line_reg.at[current_date, name_1]
        #low_predict = line_reg.at[current_date, name_2]
        #jbht_slope = slope.at[current_date, name_1]  # Коэффициент наклона
        #low_slope = slope.at[current_date, name_2]  # Коэффициент наклона

        # Расчет скользящего среднего и стандартного отклонения для спреда
        # Модель для спреда
        # model_spread = LinearRegression()
        # model_spread.fit(X[index-window:index+1], z_test['Spread'][index-window:index+1])
        # spread_predict = model_spread.predict([[numeric_date]])

        std_spread = z_test.at[current_date, 'Std_spread']
        #mean_spread = z_test.at[current_date, 'Mean_spread']
        z_score = z_test.at[current_date, 'Z-score']
        
        # Отклонение цены акции от линии регрессии для Z-score
        # Модель для JBHT
        model_jbht = LinearRegression()
        model_jbht.fit(X[index-roll_window:index+1], z_test['JBHT_Close'][index-roll_window:index+1])
        predicted_jbht = model_jbht.predict([[numeric_date]])
        # Модель для LOW
        model_low = LinearRegression()
        model_low.fit(X[index-roll_window:index+1], z_test['LOW_Close'][index-roll_window:index+1])
        predicted_low = model_low.predict([[numeric_date]])
        # Расчет отклонения от линии регрессии
        deviation_jbht = (jbht_close - predicted_jbht) / predicted_jbht
        deviation_low = (low_close - predicted_low) / predicted_low

        # Линейная регрессия для длинной и короткой позиций
        # Модель для JBHT
        model_jbht_long = LinearRegression()
        model_jbht_long.fit(X[index-window_regression:index+1], z_test['JBHT_Close'][index-window_regression:index+1])
        predicted_jbht_long = model_jbht_long.predict([[numeric_date]])
        # Модель для LOW
        model_low_long = LinearRegression()
        model_low_long.fit(X[index-window_regression:index+1], z_test['LOW_Close'][index-window_regression:index+1])
        predicted_low_long = model_low_long.predict([[numeric_date]])
        # Расчет отклонения от линии регрессии
        #deviation_jbht_long = (jbht_price - predicted_jbht_long) / predicted_jbht_long
        #deviation_low_long = (low_price - predicted_low_long) / predicted_low_long
        jbht_slope = model_jbht_long.coef_[0]  # Коэффициент наклона
        low_slope = model_jbht_long.coef_[0]  # Коэффициент наклона

        # Устанавливаем пороги на основе волатильности
        entry = coef_entry * std_spread
        sell = coef_sell * std_spread

        if flag == False:
            entry = 3 * coef_entry * std_spread
            sell = 3 * coef_sell * std_spread            
            if index > loss_date + 2:
                flag = True
                entry = coef_entry * std_spread
                sell = coef_sell * std_spread

        # Торговые сигналы и сделки
        # Покупка спреда
        #if flag:
        # Покупка спреда при восходящей регрессии двух тикеров
        if jbht_close > predicted_jbht_long and low_close > predicted_low_long and low_slope > coef_long and jbht_slope > coef_long:
            if low_shares == 0 and jbht_shares == 0:
                low_shares = 0.5 * cash / low_price
                jbht_shares = 0.5 * cash / jbht_price
                cash -= low_shares * low_price
                cash -= jbht_shares * jbht_price
                line_count += 1
            elif low_shares < 0 and jbht_shares > 0:
                cash += low_shares * low_price
                low_shares = 0
                line_count += 1
                jbht_shares += cash / jbht_price
                cash = 0
            elif low_shares > 0 and jbht_shares < 0:
                cash += jbht_shares * jbht_price
                jbht_shares = 0
                line_count += 1
                low_shares += cash / low_price
                cash = 0
            long_position = True

        elif (jbht_close < predicted_jbht_long or low_close < predicted_low_long) and long_position == True:
            cash += jbht_shares * jbht_price
            cash += low_shares * low_price
            jbht_shares = 0
            low_shares = 0
            long_position = False
            line_count += 1

        # Стратегия z-счет
        elif z_score < -entry and long_position == False:
            # Длинная позиция по дорогой, короткая по дешевой
            if deviation_low < deviation_jbht:
                if low_shares == 0 and jbht_shares == 0:
                    low_shares = cash / low_price
                    jbht_shares = -(cash / jbht_price)
                    cash -= low_shares * low_price
                    cash += -jbht_shares * jbht_price
                    trade_count += 2
                elif low_shares > 0 and jbht_shares < 0:
                    delta = (-cash / jbht_price - jbht_shares) * jbht_price
                    low_shares -= delta / low_price
                    jbht_shares = -(cash / jbht_price)
                    trade_count += 1
                elif low_shares < 0 and jbht_shares > 0:
                    cash += jbht_shares * jbht_price
                    cash += low_shares * low_price
                    low_shares = 0
                    jbht_shares = 0
                    low_shares = cash / low_price
                    jbht_shares = -(cash / jbht_price)
                    trade_count += 2
            # Короткая позиция по LOW, длинная по JBHT
            else: 
                if low_shares == 0 and jbht_shares == 0:
                    jbht_shares = cash / jbht_price
                    low_shares = -(cash / low_price)
                    cash -= jbht_shares * jbht_price
                    cash += -low_shares * low_price
                    trade_count += 2
                elif low_shares < 0 and jbht_shares > 0:
                    delta = (-cash / low_price - low_shares) * low_price
                    jbht_shares -= delta / jbht_price
                    low_shares = -(cash / low_price)
                    trade_count += 1
                elif low_shares > 0 and jbht_shares < 0:
                    cash += low_shares * low_price
                    cash += jbht_shares * jbht_price
                    low_shares = 0
                    jbht_shares = 0
                    jbht_shares = cash / jbht_price
                    low_shares = -(cash / low_price)
                    trade_count += 2

        # Покупка спреда
        elif z_score > entry and long_position == False:
            # Короткая позиция по LOW, длинная по JBHT
            if deviation_jbht < deviation_low:
                if low_shares == 0 and jbht_shares == 0:
                    jbht_shares = cash / jbht_price
                    low_shares = -(cash / low_price)
                    cash -= jbht_shares * jbht_price
                    cash -= low_shares * low_price
                    trade_count += 2
                elif low_shares < 0 and jbht_shares > 0:
                    delta = (-cash / low_price - low_shares) * low_price
                    jbht_shares -= delta / jbht_price
                    low_shares = -(cash / low_price)
                    trade_count += 1
                elif low_shares > 0 and jbht_shares < 0:
                    cash += low_shares * low_price
                    cash += jbht_shares * jbht_price
                    low_shares = 0
                    jbht_shares = 0
                    jbht_shares = cash / jbht_price
                    low_shares = -(cash / low_price)
                    trade_count += 2
            # Длинная позиция по LOW, короткая по JBHT
            else:
                if low_shares == 0 and jbht_shares == 0:
                    low_shares = cash / low_price
                    jbht_shares = -(cash / jbht_price)
                    cash -= low_shares * low_price
                    cash += -jbht_shares * jbht_price
                    trade_count += 2
                elif low_shares > 0 and jbht_shares < 0:
                    delta = (-cash / jbht_price - jbht_shares) * jbht_price
                    low_shares -= delta / low_price
                    jbht_shares = -(cash / jbht_price)
                    trade_count += 1
                elif low_shares < 0 and jbht_shares > 0:
                    cash += jbht_shares * jbht_price
                    cash += low_shares * low_price
                    low_shares = 0
                    jbht_shares = 0
                    low_shares = cash / low_price
                    jbht_shares = -(cash / jbht_price)
                    trade_count += 2

        # Выход из позиции по спреду
        elif -sell < z_score < sell and low_shares != 0 and long_position == False:
            cash += low_shares * low_price
            cash += jbht_shares * jbht_price
            low_shares = 0
            jbht_shares = 0
            trade_count += 2

        # Установка стоп-лосса
        # Расчет текущей стоимости позиций         
        value_low = low_shares * low_close
        value_jbht = jbht_shares * jbht_close
        total_value = cash + value_low + value_jbht

        if max_cash < cash:
            max_cash = cash
        stop_loss = coef_loss * max_cash

        # Расчет текущей позиции
        current_loss = max_cash - total_value

        # Проверка условий стоп-лосса
        if current_loss > stop_loss:
            # Закрытие всех позиций
            cash += low_shares * low_price
            cash += jbht_shares * jbht_price
            low_shares = 0
            jbht_shares = 0
            trade_count += 1
            max_cash = cash
            flag = False
            loss_date = index
                
        # Установка take-profit
        stop_profit = coef_profit * max_cash

        # Расчет текущего профита
        current_profit = total_value - max_cash

        if current_profit > stop_profit:
            # Закрытие всех позиций
            cash += low_shares * low_price
            cash += jbht_shares * jbht_price
            low_shares = 0
            jbht_shares = 0
            trade_count += 1

        # Если есть шорт позиции, списываем комиссию за день
        if jbht_shares < 0:
            daily_fee = -jbht_shares * jbht_price * 0.13 / 365
            cash -= daily_fee
        elif low_shares < 0:
            daily_fee = -low_shares * low_price * 0.13 / 365
            cash -= daily_fee
            
            #z_test.at[current_date, 'Cash'] = cash
                   
    return {'pair': pair, 'final_result': cash, 'Position' : total_value, 'trade_count': trade_count, 'line_count' : line_count}

In [956]:
def pre_parameters(pair):
    # Стартовые параметры
    # Оптимальные параметры:  [20.12715593  8.65949107  0.14615061 12.60124853 31.07073092  1.38127755, 0.15555868  0.14070436]
    # Определите диапазоны для каждого параметра
    window = 28
    window_regression = 9
    coef_long = 0.3
    roll_window = 20
    coef_entry = 33
    coef_sell = 1
    coef_loss = 0.01
    coef_profit = 0.1
    options = [window, window_regression, coef_long, roll_window, coef_entry, coef_sell, coef_loss, coef_profit]
    window_range = np.arange(20, 33, 1)
    coef_long_range = np.arange(0, 0.55, 0.05)
    coef_entry_range = np.arange(28, 40, 1)
    best_score = 0
    
    for i in window_range:
        options[0] = i
        iteraciya = pairs_trading_heavy(pair, close_data, open_data, options)
        score = iteraciya['Position']
        # Сохраняем параметры, если текущий результат лучше предыдущего
        if score >= best_score:
            best_score = score
            window = i
            best_iter = iteraciya      
    options[0] = window

    # for j in coef_long_range:
    #     options[2] = j
    #     score = pairs_trading_heavy(pair, close_data, open_data, options)['Position']
    #     # Сохраняем параметры, если текущий результат лучше предыдущего
    #     if score >= best_score:
    #         best_score = score
    #         coef_long = j               
    # options[2] = coef_long
    
    # for i in coef_entry_range:
    #     options[4] = i 
    #     iteraciya = pairs_trading_heavy(pair, close_data, open_data, options)
    #     score = iteraciya['Position']
    #     # Сохраняем параметры, если текущий результат лучше предыдущего
    #     if score >= best_score:
    #         best_score = score
    #         coef_entry = i
    #         best_iter = iteraciya      
    # options[4] = coef_entry


    return {'pair' : pair, 'cash' : best_score, 'trade_count' : best_iter['trade_count'], 'line_count' : best_iter['line_count'], 'Params' : options}

In [ ]:
def parameters(pair, close_data, open_data):
    # Стартовые параметры
    # Оптимальные параметры:  [20.12715593  8.65949107  0.14615061 12.60124853 31.07073092  1.38127755, 0.15555868  0.14070436]
    window = 30
    window_regression = 9
    coef_long = 0.35
    roll_window = 21
    coef_entry = 35
    coef_sell = 1
    coef_loss = 0.08
    coef_profit = 0.1
    options = [window, window_regression, coef_long, roll_window, coef_entry, coef_sell, coef_loss, coef_profit, coef_short]
    # Определите диапазоны для каждого параметра
    window_range = np.arange(20, 36, 1)
    roll_window_range = np.arange(7, 28, 1)
    window_regression_range = np.arange(5, 15, 1)
    coef_long_range = (0, 0.6, 0.05)
    coef_entry_range = np.arange(25, 40, 1)
    coef_sell_range = np.arange(1, 4, 0.5)
    coef_loss_range = np.arange(0.02, 0.16, 0.01)
    coef_profit_range = np.arange(0.06, 0.2, 0.02)
    best_score = 0
    
    for i in window_range:
        for j in roll_window_range:
            if i >= j:
            # Вызываем функцию торговли с текущими параметрами
                options[0] = i
                options[3] = j
                score = pairs_trading_heavy(pair, close_data, open_data, options)['Position']

                # Сохраняем параметры, если текущий результат лучше предыдущего
                if score >= best_score:
                    best_score = score
                    window = i
                    roll_window = j

    options[0] = window
    options[3] = roll_window

    for i in window_regression_range:
        for j in coef_long_range:
            if i <= window:
                options[1] = i
                options[2] = j
                # Вызываем функцию торговли с текущими параметрами
                score = pairs_trading_heavy(pair, close_data, open_data, options)['Position']
                # Сохраняем параметры, если текущий результат лучше предыдущего
                if score >= best_score:
                    best_score = score
                    window_regression = i
                    coef_long = j
                    
    options[1] = window_regression
    options[2] = coef_long
    
    for i in coef_entry_range:
        for j in coef_sell_range:
            options[4] = i 
            options[5] = j
            # Вызываем функцию торговли с текущими параметрами
            score = pairs_trading_heavy(pair, close_data, open_data, options)['Position']

            # Сохраняем параметры, если текущий результат лучше предыдущего
            if score >= best_score:
                best_score = score
                coef_entry = i
                coef_sell = j

    options[4] = coef_entry
    options[5] = coef_sell

    for i in coef_loss_range:
        for j in coef_profit_range:
            options[6] = i
            options[7] = j
            # Вызываем функцию торговли с текущими параметрами
            iteraciya = pairs_trading_heavy(pair, close_data, open_data, options)
            score = iteraciya['Position']

            # Сохраняем параметры, если текущий результат лучше предыдущего
            if score >= best_score:
                best_score = score
                coef_loss = i
                coef_profit = j
                best_iter = iteraciya
    
    options[6] = coef_loss
    options[7] = coef_profit

    return best_iter, options

In [800]:
def parametrs_rois(x):
    window, coef_long, coef_entry, coef_sell, coef_loss, coef_profit = x

    window_regression = 6
    roll_window = 14
    close_data = data_6m.copy()
    open_data = data_6m_open.copy()
    # Проверка условий
    # if roll_window > window or window_regression > window:
    #     return float('inf')  # Возвращаем бесконечность, если условие нарушено
    options = [round(window), round(window_regression), round(coef_long, 3), round(roll_window), round(coef_entry, 2), round(coef_sell, 2), round(coef_loss, 3), round(coef_profit, 3)]
    
    result = pairs_trading_heavy(pair, close_data, open_data, options)  
    score = result['Position']  # Пример использования возвращаемого значения
    # PSO минимизирует функцию, поэтому, если нужно максимизировать score, возвращаем -score
    return -score

In [801]:
def run_pso(pair):
    lb = [20, 0, 25, 0.5, 0.02, 0.05]  # Нижние границы
    ub = [35, 0.5, 45, 4, 0.2, 0.2]  # Верхние границы
    xopt, fopt = pso(parametrs_rois, lb, ub, swarmsize=100, maxiter=200)
    return {'pair': pair, 'cash': -fopt, 'params': xopt}

In [802]:
run_pso(('LYV', 'MAA'))

Stopping search: maximum iterations reached --> 200


{'pair': ('LYV', 'MAA'),
 'cash': 118066.53374571295,
 'params': array([28.98783961,  0.09552938, 25.        ,  3.75604674,  0.08613964,
         0.13067764])}

In [343]:
run_pso(('APA', 'MCK'))

Stopping search: maximum iterations reached --> 30


{'pair': ('APA', 'MCK'),
 'cash': 119075.03346602521,
 'params': array([33.29624318,  0.44089028, 35.27697786,  1.50252875,  0.06391703,
         0.13573274])}

In [749]:
close_data = data_2022.copy()
open_data = data_2022_open.copy()
[29, 8, 0.5, 14, 34, 1, 0.1, 0.14]
window = 23
window_regression = 7
coef_long = 0.1
roll_window = 14
coef_entry = 35
coef_sell = 1
coef_loss = 0.1
coef_profit = 0.2
options = [window, window_regression, coef_long, roll_window, coef_entry, coef_sell, coef_loss, coef_profit]
pre_parameters(('ETSY', 'ROST'))

{'pair': ('ETSY', 'ROST'),
 'cash': 267556.7432584766,
 'trade_count': 49,
 'line_count': 20,
 'Params': [23, 7, 0.30000000000000004, 14, 34, 1, 0.1, 0.2]}

In [755]:
close_data = data_2022.copy()
open_data = data_2022_open.copy()
[23, 7, 0.1, 14, 35, 1, 0.1, 0.2]
window = 23
window_regression = 7
coef_long = 0.1
roll_window = 14
coef_entry = 35
coef_sell = 1
coef_loss = 0.1
coef_profit = 0.2
options = [window, window_regression, coef_long, roll_window, coef_entry, coef_sell, coef_loss, coef_profit]
pairs_trading(('ETSY', 'ROST'), close_data, open_data, options)

{'pair': ('ETSY', 'ROST'),
 'final_result': 218111.2332386399,
 'Position': 222572.98251618582,
 'trade_count': 41,
 'line_count': 24}

In [960]:
close_data = data_2022.copy()
open_data = data_2022_open.copy()
# [26, 8, 0.35000000000000003, 14, 29, 1, 0.1, 0.1]
#          window, coef_long, coef_entry, coef_sell, coef_loss, coef_profit
window = 28
window_regression = 9
coef_long = 0.3
roll_window = 20
coef_entry = 33
coef_sell = 1
coef_loss = 0.01
coef_profit = 0.1
options = [window, window_regression, coef_long, roll_window, coef_entry, coef_sell, coef_loss, coef_profit]
pairs_trading_heavy(('APTV', 'BXP'), close_data, open_data, options)

{'pair': ('APTV', 'BXP'),
 'final_result': 0.0,
 'Position': 140388.6446020962,
 'trade_count': 93,
 'line_count': 29}

In [793]:
close_data = data_2023.copy()
open_data = data_2023_open.copy()
[26, 6, 0.5, 14, 37, 1, 0.08, 0.1]
window = 21
window_regression = 6
coef_long = 0.5
roll_window = 14
coef_entry = 30
coef_sell = 1
coef_loss = 0.08
coef_profit = 0.1
options = [window, window_regression, coef_long, roll_window, coef_entry, coef_sell, coef_loss, coef_profit]
pairs_trading(('LYV', 'MAA'), close_data, open_data, options)

{'pair': ('LYV', 'MAA'),
 'final_result': 73886.67821012232,
 'Position': 71460.11546345412,
 'trade_count': 194,
 'line_count': 0}

In [ ]:
close_data = data_2022.copy()
open_data = data_2022_open.copy()
pre_results = Parallel(n_jobs=-1)(delayed(run_pso)(pair) for pair in filtred_pairs[])
df_pre = pd.DataFrame(pre_results)
df_pre = df_pre.sort_values(by='cash', ascending=False)
df_pre[:20]

In [961]:
df_pre['pair'][:20]

204      (APTV, BXP)
495      (CHTR, RCL)
942       (GE, LRCX)
171      (AMZN, KMX)
798       (DVA, HAS)
1122     (JBHT, NVR)
859       (EMN, NOW)
370      (BKNG, STT)
1478     (PODD, XYL)
919      (EXPD, SHW)
515       (CMG, FDS)
715       (DIS, RCL)
1188      (LEN, ROK)
407       (BXP, EMN)
257       (AVY, DRI)
372      (BKNG, TDG)
1271    (LRCX, POOL)
295       (AXP, CDW)
190      (AON, ODFL)
326      (BBY, TTWO)
Name: pair, dtype: object

In [970]:
results = []
close_data = data_2023_2.copy()
open_data = data_2023_2_open.copy()
for pair in df_pre['pair'][:20]:
    options = df_pre.loc[df_pre['pair'] == pair, 'Params']
    results.append(pairs_trading_heavy(pair, close_data, open_data, options))
results = pd.DataFrame(results)
results = results.sort_values(by='cash', ascending=False)
results[:20]

KeyError: 0

In [957]:
close_data = data_2m.copy()
open_data = data_2m_open.copy()
pre_results = Parallel(n_jobs=-1)(delayed(pre_parameters)(pair) for pair in filtred_pairs)
df_pre = pd.DataFrame(pre_results)
df_pre = df_pre.sort_values(by='cash', ascending=False)
df_pre[:20]

,pair,cash,trade_count,line_count,Params
204,"(APTV, BXP)",146602.255036,23,7,"[28, 9, 0.3, 20, 33, 1, 0.01, 0.1]"
495,"(CHTR, RCL)",142663.470714,17,8,"[22, 9, 0.3, 20, 33, 1, 0.01, 0.1]"
942,"(GE, LRCX)",134797.848432,14,4,"[24, 9, 0.3, 20, 33, 1, 0.01, 0.1]"
171,"(AMZN, KMX)",132221.236096,14,3,"[32, 9, 0.3, 20, 33, 1, 0.01, 0.1]"
798,"(DVA, HAS)",131889.309810,15,2,"[32, 9, 0.3, 20, 33, 1, 0.01, 0.1]"
1122,"(JBHT, NVR)",131483.274938,31,6,"[22, 9, 0.3, 20, 33, 1, 0.01, 0.1]"
859,"(EMN, NOW)",131338.306842,19,2,"[20, 9, 0.3, 20, 33, 1, 0.01, 0.1]"
370,"(BKNG, STT)",130995.680472,18,10,"[22, 9, 0.3, 20, 33, 1, 0.01, 0.1]"
1478,"(PODD, XYL)",130592.513816,23,2,"[26, 9, 0.3, 20, 33, 1, 0.01, 0.1]"
919,"(EXPD, SHW)",130533.509754,15,2,"[26, 9, 0.3, 20, 33, 1, 0.01, 0.1]"


In [621]:
df_pre[:1]["Params"]

421    [31, 7, 0.30000000000000004, 10, 30, 1, 0.02, ...
Name: Params, dtype: object

In [329]:
close_data = data_2022.copy()
open_data = data_2022_open.copy()
pre_results = Parallel(n_jobs=-1)(delayed(run_pso)(pair) for pair in df_pre['pair'][:100])
df_pso = pd.DataFrame(pre_results)
df_pso = df_pso.sort_values(by='cash', ascending=False)
df_pso[:20]

NameError: name 'df_so' is not defined

In [300]:
pre_result = []
window = 27
window_regression = 8
coef_long = 0.35
roll_window = 14
coef_entry = 35
coef_sell = 1
coef_loss = 0.1
coef_profit = 0.14
options = [window, window_regression, coef_long, roll_window, coef_entry, coef_sell, coef_loss, coef_profit]

pre_parameters(('JBHT', 'LOW'), close_data, open_data)

{'Iter': {'pair': ('JBHT', 'LOW'),
  'final_result': 191149.7901015409,
  'Position': 195959.94237062783,
  'trade_count': 107,
  'line_count': 36},
 'Params': [20, 8, 0.25, 14, 39, 1, 0.1, 0.14]}

In [ ]:
pre_result = []
window = 27
window_regression = 8
coef_long = 0.35
roll_window = 14
coef_entry = 35
coef_sell = 1
coef_loss = 0.1
coef_profit = 0.14
options = [window, window_regression, coef_long, roll_window, coef_entry, coef_sell, coef_loss, coef_profit, coef_short]

for pair in filtred_pairs:
    pre_result.append(pre_parameters(pair, close_data, open_data))
df_pre = pd.DataFrame(results)
df_pre = df_pre.sort_values(by='Position', ascending=False)
df_pre[:20]

In [149]:
results = []
close_data = data_2022.copy()
open_data = data_2022_open.copy()
window = 27
window_regression = 10
coef_long = 0.15
roll_window = 14
coef_entry = 35
coef_sell = 1
coef_loss = 0.1
coef_profit = 0.12
options = [window, window_regression, coef_long, roll_window, coef_entry, coef_sell, coef_loss, coef_profit]
for pair in filtred_pairs:
    results.append(pairs_trading(pair, close_data, open_data, options))
# Создание DataFrame из списка словарей
df_start = pd.DataFrame(results)
df_start = df_start.sort_values(by='Position', ascending=False)
df_start[:20]

,pair,final_result,Position,trade_count,line_count
5905,"(BALL, SWK)",3.178728e+05,318867.640704,102,14
20553,"(IRM, KR)",3.062347e+05,306689.679016,89,6
6739,"(BIIB, TJX)",2.798298e+05,289646.418013,119,19
4156,"(APA, LMT)",3.057428e+05,287277.746315,58,16
24004,"(MRNA, ORCL)",2.800563e+05,280056.341703,39,20
14043,"(EA, FDX)",2.638260e+05,278346.695439,107,22
527,"(ABNB, DVA)",2.716661e+05,271666.057357,37,18
9534,"(CHRW, INVH)",2.630141e+05,268330.772427,117,6
23376,"(META, NTRS)",2.683778e+05,260990.409272,82,16
12616,"(DAY, PNC)",2.609263e+05,260926.322576,60,23


In [245]:
df_pso = pd.DataFrame(result)
df_pso = df_pso.sort_values(by='cash', ascending=False)
df_pso[:20]

,pair,cash,params
3,"(A, AMP)",189776.623630,"[25.234164405451722, 13.425021387941344, 0.377..."
10,"(A, BSX)",187539.143275,"[25.05790343924349, 12.843483622677542, 0.4957..."
12,"(A, CAG)",186434.140858,"[25.22365699486485, 14.763584532253896, 0.3478..."
20,"(A, DHI)",185278.437234,"[34.64591878963863, 20.0, 0.5, 5.0, 25.0, 0.51..."
15,"(A, CDW)",182705.926895,"[25.446630950972015, 19.506719948877404, 0.411..."
22,"(A, DRI)",182286.790518,"[20.0, 18.930289259122326, 0.12400446894140056..."
17,"(A, CTAS)",180843.608537,"[32.8275942750053, 15.967545179974131, 0.07245..."
9,"(A, BIIB)",174858.090130,"[33.386565382958636, 14.561746216016502, 0.069..."
0,"(A, ACGL)",174742.177934,"[32.832576322204524, 19.749602461371957, 0.225..."
19,"(A, DGX)",174451.672665,"[29.102749643522362, 16.234238033078125, 0.071..."


In [150]:
close_data = data_2022.copy()
open_data = data_2022_open.copy()
window = 27
window_regression = 8
coef_long = 0.15
roll_window = 14
coef_entry = 35
coef_sell = 1
coef_loss = 0.1
coef_profit = 0.12
options = [window, window_regression, coef_long, roll_window, coef_entry, coef_sell, coef_loss, coef_profit]
pairs_trading_heavy(('ETN', 'AMAT'), close_data, open_data, options)

{'pair': ('ETN', 'AMAT'),
 'final_result': 127594.13182660208,
 'Position': 132240.37532210723,
 'trade_count': 87,
 'line_count': 34}

In [151]:
pairs_trading(('ETN', 'AMAT'), close_data, open_data, options)

{'pair': ('ETN', 'AMAT'),
 'final_result': 134936.99597396434,
 'Position': 139850.62429582988,
 'trade_count': 90,
 'line_count': 25}

In [204]:
# PSO. Определение границ параметров
lb = [20, 0, 25, 0.5, 0.02, 0.1]  # Нижние границы
ub = [35, 0.5, 45, 3, 0.2, 0.2]  # Верхние границы    
# Запуск PSO
xopt, fopt = pso(parametrs_rois, lb, ub, swarmsize=20, maxiter=30)

# Вывод результатов
print("Оптимальные параметры: ", xopt)
print("Лучшая метрика score: ", -fopt)

Stopping search: maximum iterations reached --> 30
Оптимальные параметры:  [21.73771568  0.30462031 42.22802603  1.0305328   0.15402748  0.12308428]
Лучшая метрика score:  229042.55344115518


In [205]:
test

{'params': [array([21.73771568,  0.30462031, 42.22802603,  1.0305328 ,  0.15402748,
          0.12308428])],
 'cash': 229042.55344115518}

In [ ]:
results = []
results_2 = []
for pair in pairs:
    results.append(pairs_trading(pair, options))
for pair in results[:100]['Pair']:
    results_2.append(pre_parametrs(pair))
sort results_2
for pair in results_2[:5]:
    results_3.append(parametrs((pair, options)))
sort results_3
for pair in results_3[:3]:
    final.append(parametrs_roi(results_3[0], options))
sort final
pairs_trading_heavy(final[0], options, data)

In [ ]:
message = f"Оптимальные параметры: {xopt}\nЛучшая метрика score: {-fopt}"
# Отправка сообщения
send_telegram_message(message)

In [ ]:
regression(unique_tickers, 23, 10)